<a href="https://colab.research.google.com/github/Chaotic-Legend/CMP-414-Codes/blob/main/Homework%20%232%3A%20Intelligent%20Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Homework #2: Intelligent Agent

In [ ]:
import random
from collections import deque
import heapq
# ---------------- Agent Class ----------------
class Agent:
    def __init__(self, grid, target, start=None):
        self.grid = grid
        self.num_rows = len(grid)
        self.num_cols = len(grid[0])
        self.target = target

        if start is None:
            while True:
                r = random.randint(0, self.num_rows - 1)
                c = random.randint(0, self.num_cols - 1)
                if [r, c] != target and grid[r][c] != "X":
                    start = [r, c]
                    break

        self.position = start[:]
        self.visited = [start[:]]
        self.path = [start[:]]
        self.planned_path = self._bfs_shortest_path(start, target)
        if self.planned_path is not None:
            self.mode = "optimal"
            self._next_index = 1
        else:
            self.mode = "explore"
            self._next_index = None

    # ---------- Shortest-Path (BFS) ----------
    def _bfs_shortest_path(self, start, target):
        start_t = tuple(start)
        target_t = tuple(target)
        if start_t == target_t:
            return [list(start_t)]

        q = deque([start_t])
        parent = {start_t: None}
        visited = {start_t}

        while q:
            node = q.popleft()
            if node == target_t:
                path = []
                cur = node
                while cur is not None:
                    path.append([cur[0], cur[1]])
                    cur = parent[cur]
                path.reverse()
                return path

            r, c = node
            neighbors = [
                (r-1, c), (r+1, c), (r, c-1), (r, c+1)
            ]
            for nr, nc in neighbors:
                if 0 <= nr < self.num_rows and 0 <= nc < self.num_cols:
                    if self.grid[nr][nc] != "X":
                        nt = (nr, nc)
                        if nt not in visited:
                            visited.add(nt)
                            parent[nt] = node
                            q.append(nt)
        return None

    def _astar_shortest_path(self, start, target):
        start_t = tuple(start)
        target_t = tuple(target)
        def heuristic(a, b):
            return abs(a[0]-b[0]) + abs(a[1]-b[1])

        open_heap = []
        heapq.heappush(open_heap, (heuristic(start_t, target_t), 0, start_t))
        came_from = {start_t: None}
        gscore = {start_t: 0}
        closed = set()

        while open_heap:
            f, g, current = heapq.heappop(open_heap)
            if current == target_t:
                path = []
                cur = current
                while cur is not None:
                    path.append([cur[0], cur[1]])
                    cur = came_from[cur]
                path.reverse()
                return path

            if current in closed:
                continue
            closed.add(current)

            r, c = current
            neighbors = [(r-1, c),(r+1, c),(r, c-1),(r, c+1)]
            for nt in neighbors:
                nr, nc = nt
                if 0 <= nr < self.num_rows and 0 <= nc < self.num_cols and self.grid[nr][nc] != "X":
                    tentative_g = g + 1
                    if nt not in gscore or tentative_g < gscore[nt]:
                        gscore[nt] = tentative_g
                        came_from[nt] = current
                        fscore = tentative_g + heuristic(nt, target_t)
                        heapq.heappush(open_heap, (fscore, tentative_g, nt))
        return None

    # ---------- Perception & Decision ----------
    def perceive_environment(self):
        row, col = self.position
        up = "Wall" if row == 0 else ("Obstacle" if self.grid[row-1][col] == "X" else "Empty")
        down = "Wall" if row == self.num_rows-1 else ("Obstacle" if self.grid[row+1][col] == "X" else "Empty")
        left = "Wall" if col == 0 else ("Obstacle" if self.grid[row][col-1] == "X" else "Empty")
        right = "Wall" if col == self.num_cols-1 else ("Obstacle" if self.grid[row][col+1] == "X" else "Empty")
        return up, down, left, right

    def decide_action(self):
        if self.mode == "optimal" and self.planned_path is not None:
            if self.position == self.target:
                return None
            if self._next_index < len(self.planned_path):
                next_r, next_c = self.planned_path[self._next_index]
                r, c = self.position
                dr, dc = next_r - r, next_c - c
                if dr == 1: return "Down"
                elif dr == -1: return "Up"
                elif dc == 1: return "Right"
                elif dc == -1: return "Left"
                else:
                    self.mode = "explore"
                    return self._decide_explore()
            else:
                return None
        return self._decide_explore()

    def _decide_explore(self):
        up, down, left, right = self.perceive_environment()
        row, col = self.position
        if right == "Empty" and [row, col+1] not in self.visited:
            return "Right"
        elif down == "Empty" and [row+1, col] not in self.visited:
            return "Down"
        elif up == "Empty" and [row-1, col] not in self.visited:
            return "Up"
        elif left == "Empty" and [row, col-1] not in self.visited:
            return "Left"
        else:
            return "Backtrack"

    # ---------- Action Execution ----------
    def take_action(self, action=None):
        if action is None:
            action = self.decide_action()
        if self.mode == "optimal" and self.planned_path is not None:
            if self.position == self.target:
                return
            if self._next_index < len(self.planned_path):
                next_pos = self.planned_path[self._next_index]
                self.position = [next_pos[0], next_pos[1]]
                self._next_index += 1
                if self.position not in self.visited:
                    self.visited.append(self.position[:])
                self.path.append(self.position[:])
            return

        row, col = self.position
        if action == "Right": col += 1
        elif action == "Down": row += 1
        elif action == "Up": row -= 1
        elif action == "Left": col -= 1
        elif action == "Backtrack":
            if len(self.path) > 1:
                self.path.pop()
                row, col = self.path[-1]
        self.position = [row, col]
        if self.position not in self.visited:
            self.visited.append(self.position[:])
        if action != "Backtrack":
            self.path.append(self.position[:])

# ---------------- Environment Creation ----------------
def create_environment(num_rows, num_cols, num_obstacles):
    grid = [["_" for _ in range(num_cols)] for _ in range(num_rows)]

    while True:
        target_row = random.randint(0, num_rows - 1)
        target_col = random.randint(0, num_cols - 1)
        if grid[target_row][target_col] == "_":
            break
    target = [target_row, target_col]
    grid[target_row][target_col] = "T"

    count = 0
    while count < num_obstacles:
        r = random.randint(0, num_rows - 1)
        c = random.randint(0, num_cols - 1)
        if grid[r][c] == "_" and [r, c] != target:
            grid[r][c] = "X"
            count += 1

    return grid, target

# ---------------- Grid Display ----------------
def print_grid(grid, agent=None):
    display = [row.copy() for row in grid]

    if agent:
        for r, c in agent.path:
            if display[r][c] == "_":
                display[r][c] = "."
        start_r, start_c = agent.path[0]
        display[start_r][start_c] = "0"
        if agent.position == agent.target:
            r, c = agent.position
            display[r][c] = "A"

    for row in display:
        print(" ".join(row))
    print()

# ---------------- Simulation Example ----------------
if __name__ == "__main__":
    grid, target_pos = create_environment(6, 9, 10)
    agent = Agent(grid, target=target_pos)

    while agent.planned_path is None:
        grid, target_pos = create_environment(6, 9, 10)
        agent = Agent(grid, target=target_pos)

    print("Initial Grid:")
    print_grid(grid, agent)
    actions_taken = []
    positions_taken = []

    while agent.position != agent.target:
        action = agent.decide_action()
        agent.take_action(action)
        actions_taken.append(action)
        positions_taken.append(agent.position.copy())

    print("Steps To Reach Rarget:")
    print(f"Step 0 - Start At: {agent.path[0]}")
    for i, (a, p) in enumerate(zip(actions_taken, positions_taken), start=1):
        print(f"Step {i} - Action: {a}, Position: {p}")

    print(f"\nReached Target At: {agent.position}")
    print("\nVisited Cells:", agent.visited)
    print("\nPlanned/Correct Path:", agent.planned_path)
    print("\nFinal Grid:")
    print_grid(grid, agent)

Initial Grid:
X _ _ _ _ _ _ _ X
_ _ _ _ _ _ _ _ _
_ X _ _ X _ _ T _
_ _ 0 _ _ _ _ _ _
_ X X _ _ _ X _ X
_ _ _ _ X _ X _ _

Steps To Reach Rarget:
Step 0 - Start At: [3, 2]
Step 1 - Action: Right, Position: [3, 3]
Step 2 - Action: Right, Position: [3, 4]
Step 3 - Action: Right, Position: [3, 5]
Step 4 - Action: Up, Position: [2, 5]
Step 5 - Action: Right, Position: [2, 6]
Step 6 - Action: Right, Position: [2, 7]

Reached Target At: [2, 7]

Visited Cells: [[3, 2], [3, 3], [3, 4], [3, 5], [2, 5], [2, 6], [2, 7]]

Planned/Correct Path: [[3, 2], [3, 3], [3, 4], [3, 5], [2, 5], [2, 6], [2, 7]]

Final Grid:
X _ _ _ _ _ _ _ X
_ _ _ _ _ _ _ _ _
_ X _ _ X . . A _
_ _ 0 . . . _ _ _
_ X X _ _ _ X _ X
_ _ _ _ X _ X _ _

